# 1. 사전 준비 항목 (모델 및 데이터 로드)
 - 2_recommendation_loigic_test.ipynb의 모델 및 데이터 로딩 부분을 웹 데모 버전에 맞게 수정하여 사용

In [ ]:
# --- 1. 환경 설정 ---

# 1. 라이브러리 설치
!pip install -q transformers accelerate bitsandbytes
!pip install -q ftfy regex tqdm
!pip install -q git+https://github.com/openai/CLIP.git
!pip install -q librosa

# 2. 라이브러리 임포트
import os
import io
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch import nn
from PIL import Image
from typing import List
from google.colab import files
from IPython.display import Audio, display
from pprint import pprint
from transformers import (
    ClapModel, ClapProcessor,
    Blip2ForConditionalGeneration, AutoProcessor,
    InstructBlipForConditionalGeneration,
    CLIPModel, CLIPProcessor as HFCLIPProcessor
)

# 3. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

print("✅ 1. 환경 설정 완료")

In [ ]:
# --- 2. 경로 및 전역 설정 ---

# === 기본 설정 ===
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE_TORCH = torch.float16 if DEVICE == "cuda" else torch.float32
TOPK = 3  # 추천할 트랙 개수

# === 파일 경로 (사용자 환경에 맞게 수정) ===
MUSIC_EMB_PATH = "/content/drive/MyDrive/Jamendo_Embeddings_Final/song_embeds.npy"
MUSIC_META_PATH = "/content/drive/MyDrive/Jamendo_Embeddings_Final/song_paths.csv"
CLIP_EMOTION_CKPT = "/content/drive/MyDrive/image-music-recommendation-system_final/clip_emotion_classifier.pt"

# === 테스트할 이미지 경로 (사용자 환경에 맞게 수정) ===
IMAGE_PATHS = ["/content/drive/MyDrive/Datasets/EmoSet-118K/image/amusement/amusement_00000.jpg"]

# === 모델 체크포인트 ===
# 음악 임베딩 만들 때 쓴 것과 동일해야 함
CLAP_CKPT = "laion/clap-htsat-unfused"

# 짧은 캡션
BLIP2_CKPT = "Salesforce/blip2-flan-t5-xl"

# 긴 설명
INSTRUCTBLIP_CKPT = "Salesforce/instructblip-flan-t5-xl"

# 감정 추출 CLIP 모델
CLIP_CHECKPOINT = "openai/clip-vit-base-patch32"

# === 감정 클래스 라벨 ===
EMO_LABELS = ["amusement", "anger", "awe", "contentment", "disgust", "excitement", "fear", "sadness"]

print(f"✅ 2. 설정 완료. Device: {DEVICE}, DType: {DTYPE_TORCH}")

In [ ]:
# --- 3. 모든 AI 모델 로드 ---

# (1) CLAP
clap_model = ClapModel.from_pretrained(CLAP_CKPT).to(DEVICE).eval()
clap_processor = ClapProcessor.from_pretrained(CLAP_CKPT)
print("✅ CLAP 로드 완료.")

# (2) BLIP-2
blip2_processor = AutoProcessor.from_pretrained(BLIP2_CKPT)
blip2_model = Blip2ForConditionalGeneration.from_pretrained(BLIP2_CKPT, torch_dtype=DTYPE_TORCH, device_map="auto").eval()
print("✅ BLIP-2 로드 완료.")

# (3) InstructBLIP
instruct_processor = AutoProcessor.from_pretrained(INSTRUCTBLIP_CKPT)
instruct_model = InstructBlipForConditionalGeneration.from_pretrained(INSTRUCTBLIP_CKPT, torch_dtype=DTYPE_TORCH, device_map="auto").eval()
print("✅ InstructBLIP 로드 완료.")

# (4) Fine-tuned CLIP Emotion Classifier
class CLIPEmotionHead(nn.Module):
    def __init__(self, in_dim, num_classes):
        super().__init__()
        self.classifier = nn.Linear(in_dim, num_classes)
    def forward(self, x): return self.classifier(x)

# --- CLIP 백본 로드 ---
clip_backbone = CLIPModel.from_pretrained(CLIP_CHECKPOINT, torch_dtype=DTYPE_TORCH).to(DEVICE).eval()
clip_proc = HFCLIPProcessor.from_pretrained(CLIP_CHECKPOINT)

# --- Emotion Head 인스턴스 생성 및 가중치 로드 ---
IN_DIM = clip_backbone.config.projection_dim
emotion_head = CLIPEmotionHead(in_dim=IN_DIM, num_classes=len(EMO_LABELS)).to(DEVICE, dtype=DTYPE_TORCH).eval()

# 체크포인트(.pth)에서 가중치를 그대로 로드
ckpt = torch.load(CLIP_EMOTION_CKPT, map_location=DEVICE)

if "classifier.weight" in ckpt:
    # 모델 전체 또는 헤드 전체가 저장된 경우
    final_state_dict = {k: v for k, v in ckpt.items() if k.startswith("classifier.")}
    emotion_head.load_state_dict(final_state_dict)
else:
    # 분류기 부분만 저장된 경우 (키에 'classifier.' 접두사가 없음)
    final_state_dict = {k: v for k, v in ckpt.items() if k.startswith("classifier.")}
    if not final_state_dict:
        try:
             emotion_head.classifier.load_state_dict(ckpt)
        except RuntimeError:
             final_state_dict = {k.replace("classifier.", ""): v for k, v in ckpt.items() if "classifier." in k}
             emotion_head.classifier.load_state_dict(final_state_dict)

    else:
        emotion_head.load_state_dict(final_state_dict)
print("✅ CLIP 로드 완료.")

print("✅ 3. 모든 모델 로드 완료")

In [ ]:
# --- 4. 음악 DB 로드 및 핵심 로직 함수 정의 ---

# (1) 음악 DB 로드
song_embeds = torch.from_numpy(np.load(MUSIC_EMB_PATH)).float().to(DEVICE)
song_meta = pd.read_csv(MUSIC_META_PATH)
song_embeds = song_embeds / song_embeds.norm(dim=-1, keepdim=True)
print(f"✅ 음악 DB 로드 완료: {len(song_meta)} 곡")

# (2) AI 파이프라인 함수들
@torch.inference_mode()
def predict_emotion(image):
    # processor 출력도 모델 dtype으로 캐스팅
    inputs = clip_proc(images=image, return_tensors="pt").to(DEVICE, dtype=DTYPE_TORCH)

    # CLIP 비주얼 임베딩 (dtype 일치)
    img_feats = clip_backbone.get_image_features(**inputs)    # [1, D], dtype=DTYPE_TORCH

    # 로짓 → 온도 스케일 → 소프트맥스
    logits = emotion_head(img_feats)                          # [1, C], dtype=DTYPE_TORCH
    probs = torch.softmax(logits.float(), dim=-1)[0]          # 안정성 위해 softmax만 float32

    top_id = torch.argmax(probs).item()
    return {
        "label": EMO_LABELS[top_id],
        "prob": probs[top_id].item(),
        "all_probs": probs.cpu().numpy()
    }

@torch.inference_mode()
def blip2_caption(image):
    inputs = blip2_processor(images=image, return_tensors="pt").to(DEVICE, DTYPE_TORCH)
    out = blip2_model.generate(**inputs, max_new_tokens=30)
    return blip2_processor.batch_decode(out, skip_special_tokens=True)[0].strip()

@torch.inference_mode()
def instructblip_describe(image):
    prompt = "Write a detailed, vivid description of the image focusing on mood, scene, colors, lighting, and context."
    inputs = instruct_processor(images=image, text=prompt, return_tensors="pt").to(DEVICE, DTYPE_TORCH)
    out = instruct_model.generate(**inputs, max_new_tokens=160)
    return instruct_processor.batch_decode(out, skip_special_tokens=True)[0].strip()

def build_texts_for_image(pil_img):
    cap_short = blip2_caption(pil_img)
    emo = predict_emotion(pil_img)
    cap_combo = f"This image conveys {emo['label']} and feels like {emo['label']} mood. Caption: {cap_short}"
    cap_long = instructblip_describe(pil_img)
    return cap_short, cap_combo, cap_long, emo

@torch.inference_mode()
def score_all_songs_with_text(text: str):
    inputs = clap_processor(text=[text], return_tensors="pt", padding=True).to(DEVICE)
    text_emb = clap_model.get_text_features(**inputs)
    text_emb = text_emb / text_emb.norm(dim=-1, keepdim=True)
    sims = text_emb @ song_embeds.T
    return sims.squeeze(0)

def topk_emotions_from_probs(prob_array, labels, k=3):
    idx = np.asarray(prob_array).argsort()[::-1][:k]
    return [(labels[i], float(prob_array[i])) for i in idx]

def show_topk_audio(sims_1d, k=3, title="Results"):
    sims_cpu = sims_1d.detach().float().cpu().numpy()
    idx = sims_cpu.argsort()[::-1][:k]

    print(f"\n== {title} (Top-{k}) ==")

    for r, i in enumerate(idx, start=1):
        score = float(sims_cpu[i])
        row = song_meta.iloc[i]

        full_path = row.get("path", "N/A")

        try:
            dir_path, file_name = os.path.split(full_path)
            parent_dir_path, dir_name = os.path.split(dir_path)
            meaningful_path = f"{dir_name}/{file_name}"
        except:
            meaningful_path = full_path # 경로 파싱 실패 시 원본 경로 표시

        print(f"[{r:02d}] score={score:.3f} | title='{meaningful_path}'")

        try:
            display(Audio(filename=full_path))
        except Exception as e:
            print(f"  (Audio preview failed for path: {full_path}. Error: {e})")

@torch.no_grad()
def clap_text_embed(texts: List[str]) -> torch.Tensor:
    inputs = clap_processor(text=texts, return_tensors="pt", padding=True, truncation=True).to(DEVICE)
    text_feats = clap_model.get_text_features(**inputs)  # [B, D]
    return l2norm(text_feats)

# (3) 메인 데모 함수
def demo_recommend_3plus3(pil_image, lam=0.6, top_k=3):
    # 텍스트 쿼리 3종 생성
    cap_short, cap_combo, cap_long, emo = build_texts_for_image(pil_image)

    # 결과 헤더 출력
    print("="*100)
    print(f"[IMAGE ANALYSIS RESULTS]")
    # top-3 감정 포매팅
    emo_top3 = topk_emotions_from_probs(emo["all_probs"], EMO_LABELS, k=3)
    print("CLIP Emotions (Top-3):", ", ".join([f"{e}({p:.2f})" for e,p in emo_top3]))
    print(f"BLIP-2 Caption: {cap_short}")
    print(f"InstructBLIP Caption (Long): {cap_long}")
    print("="*100)

    # Stage-1 추천 (BLIP2+CLIP 융합 쿼리)
    s1 = score_all_songs_with_text(cap_combo)
    show_topk_audio(s1, k=top_k, title="▶ Stage-1 Recommendations (Emotion + Facts)")

    # Stage-2 재랭킹 (InstructBLIP 상세 묘사 추가)
    s2 = score_all_songs_with_text(cap_long)
    fused_scores = lam * s1 + (1.0 - lam) * s2
    show_topk_audio(fused_scores, k=top_k, title=f"▶ Stage-2 Re-ranked Recommendations (Context Enhanced)")

print("✅ 4. 핵심 로직 함수 정의 완료")

# 2. 실제 웹 데모 실행 부분

In [ ]:
# --- 1. 웹 서버 및 터널링 라이브러리 준비 ---

# 1. 라이브러리 설치
!pip install -q fastapi uvicorn[standard] python-multipart nest_asyncio starlette cloudflared

# 2. 라이브러리 임포트
import nest_asyncio
import uvicorn
import threading
import time
import re
import subprocess

from fastapi import FastAPI, UploadFile, File, Response
from fastapi.responses import HTMLResponse, JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles

# 3. 비동기 환경 설정
# Colab/Jupyter 환경에서 uvicorn 서버를 실행하기 위해 필요
nest_asyncio.apply()

# 4. 전역 및 기타 설정
AUDIO_ROOT = "/content/drive/MyDrive/Datasets"
assert os.path.exists(AUDIO_ROOT), f"AUDIO_ROOT not found: {AUDIO_ROOT}"

# CSV 컬럼명 유연하게 찾기
PATH_COL = next((c for c in ["path","filepath","file"] if c in song_meta.columns), song_meta.columns[0])
TITLE_COL = next((c for c in ["title","name"] if c in song_meta.columns), None)

print("✅ 1. 웹 데모 환경 설정 완료.")

In [ ]:
# --- 2. HTML UI 정의 ---
# 간단한 이미지 업로드 및 결과 표시 페이지
INDEX_HTML = """
<!doctype html><html lang="ko"><head>
<meta charset="utf-8"/><meta name="viewport" content="width=device-width,initial-scale=1"/>
<title>사진-음악 추천</title>
<link rel="icon" type="image/png" href="/static/icon4.png">
<style>
 body{
   font-family:system-ui,sans-serif;
   max-width:920px;margin:0 auto;padding:0;
   background:#f9fafc;
 }
 header{
   background:#002b5b;
   color:white;
   padding:20px 24px;
   border-radius:0 0 20px 20px;
   text-align:center;
 }
 .card{
   background:white;
   border:1px solid #ddd;
   border-radius:14px;
   padding:16px;
   margin:16px 0;
   box-shadow:0 2px 6px rgba(0,0,0,0.08);
 }
 .song{
   border:1px solid #eee;
   border-radius:12px;
   padding:10px;
   margin:8px 0;
 }
 .muted{color:#666;font-size:14px}
 button{
   padding:10px 16px;
   border:1px solid #ccc;
   border-radius:10px;
   cursor:pointer;
   background:#004080;
   color:white;
 }
 button:hover{
   background:#0055aa;
 }
 #preview{
   max-width:280px;
   border:1px solid #eee;
   border-radius:10px;
   margin-top:10px;
 }
 footer{
   text-align:center;
   padding:20px;
   font-size:14px;
   color:#888;
   margin-top:40px;
   border-top:1px solid #ddd;
 }
</style>
</head><body>
<header>
  <h1> AI 이미지 음악 추천 시스템 </h1>
  <p>HKNU AI, Music and Photography</p>
</header>

<div class="card">
  <input id="file" type="file" accept="image/*"/>
  <button id="btn" type="button">추천 받기</button>
  <span id="msg" class="muted"></span>
  <div style="margin-top:10px; color:#aa0000; font-size:14px;">
    ⚠️ 업로드된 이미지는 서버에 저장되지 않습니다.
  </div>
  <div><img id="preview" src=""/></div>
</div>

<div class="card">
  <h3>요약</h3>
  <div id="summary"></div>
</div>

<div class="card">
  <h3>1단계: CLIP + BLIP2 이미지 분위기 기반 음악 추천 (Top-3)</h3>
  <div id="stage1"></div>
</div>

<div class="card">
  <h3>2단계: InstructBLIP 설명 확장 기반 재추천 (Top-3)</h3>
  <div id="stage2"></div>
</div>

<footer>© HKNU AI, Music and Photography </footer>

<script>
const $ = id => document.getElementById(id);
const file = $("file"), btn = $("btn"), preview = $("preview"), msg = $("msg");
const summary = $("summary"), stage1Box = $("stage1"), stage2Box = $("stage2");

file.addEventListener("change", () => {
  const f = file.files[0];
  if (!f) { preview.src=""; msg.textContent=""; return; }
  preview.src = URL.createObjectURL(f);
  msg.textContent = `${f.name} (${Math.round(f.size/1024)} KB)`;
});

async function safeJson(r){
  const txt = await r.text();
  try { return JSON.parse(txt); } catch { return { ok:false, error: txt }; }
}

btn.addEventListener("click", async (e) => {
  e.preventDefault();
  const f = file.files[0];
  if (!f) { alert("이미지를 선택하세요."); return; }

  summary.innerHTML = "분석 중…";
  stage1Box.innerHTML = "";
  stage2Box.innerHTML = "";

  const fd = new FormData();
  fd.append("file", f, f.name);

  try {
    const r = await fetch(location.origin + "/recommend", { method:"POST", body: fd });
    const data = await safeJson(r);
    if (!r.ok || !data.ok){
      const m = (data && data.error) ? `${data.error} (stage=${data.stage||"?"})` : `HTTP ${r.status}`;
      alert("서버 오류: " + m);
      console.error("server error:", data);
      return;
    }

    summary.innerHTML = `
      <div><b>Emotion:</b> ${data.emotion.label} (p=${data.emotion.prob.toFixed(3)})</div>
      <div><b>BLIP2 caption:</b> ${data.texts.blip2_caption}</div>
      <div><b>BLIP2+CLIP caption:</b> ${data.texts.clip_plus_blip2}</div>
      <div><b>InstructBLIP expanded:</b> ${data.texts.instruct_expanded}</div>
    `;

    const render = arr => arr.map(x => `
      <div class="song">
        <div><b>${x.title || "(no title)"}</b></div>
        <div class="muted">score=${x.score.toFixed(4)}</div>
        ${x.preview_url ? `<audio controls src="${x.preview_url}"></audio>` : `<div class="muted">미리듣기 URL 없음</div>`}
        <div class="muted">${x.path}</div>
      </div>
    `).join("");

    stage1Box.innerHTML = render(data.stage1);
    stage2Box.innerHTML = render(data.stage2);

  } catch (e) {
    console.error(e);
    alert("요청 실패: " + e.message);
  }
});
</script>
</body></html>
"""

print("✅ 2. HTML UI 정의 완료.")

In [ ]:
# --- 3. 웹 서버용 헬퍼 함수 정의 ---

# --- 헬퍼 함수 ---
def l2norm(x: torch.Tensor, eps: float = 1e-8):
    return x / (x.norm(dim=-1, keepdim=True) + eps)

def _to_py(x):
    if isinstance(x, (np.floating,)): return float(x)
    if isinstance(x, (np.integer,)):  return int(x)
    if isinstance(x, (np.bool_,)):    return bool(x)
    return x

def to_media_url(path: str) -> str:
  if re.match(r"^https?://", str(path)):
        return str(path)
  p = os.path.abspath(str(path))
  root = os.path.abspath(AUDIO_ROOT)
  if p.startswith(root):
      rel = p[len(root):].lstrip(os.sep)
      return f"/media/{rel}"
  return ""

@torch.inference_mode()
def score_all_songs_with_text(text: str) -> torch.Tensor:
    emb = clap_text_embed([text]).to(song_embeds.device)
    emb = emb / (emb.norm(dim=-1, keepdim=True) + 1e-8)
    sims = emb @ song_embeds.T          # [1,N]
    return sims.squeeze(0)              # [N]

def topk_block(scores_1d: torch.Tensor, k: int = 3):
    sims = scores_1d.detach().float().cpu().numpy()
    idx = sims.argsort()[::-1][:k]
    out = []
    for r, i in enumerate(idx, start=1):
        score = float(sims[i])
        row = song_meta.iloc[i]
        path = str(row[PATH_COL])
        if TITLE_COL is None or pd.isna(row.get(TITLE_COL, None)):
            base = os.path.basename(path)
            title = os.path.splitext(base)[0]
        else:
            title = str(row[TITLE_COL])
        out.append({
            "rank": r,
            "index": int(i),
            "score": score,
            "title": title,
            "path": path,
            "preview_url": to_media_url(path),
        })
    return out

print("✅ 3. 웹 서버용 헬퍼 함수 정의 완료.")

In [ ]:
# --- 4. FastAPI 웹 서버 및 Cloudflare Tunnel 실행 ---

app = FastAPI(title="Image→Music Recommendation API")
app.add_middleware(CORSMiddleware, allow_origins=["*"], allow_credentials=True, allow_methods=["*"], allow_headers=["*"])
app.mount("/media", StaticFiles(directory=AUDIO_ROOT), name="media")

@app.exception_handler(Exception)
async def global_exception_handler(request, exc):
    import traceback; traceback.print_exc()
    return JSONResponse(status_code=500, content={"ok": False, "error": str(exc)})

@app.get("/", response_class=HTMLResponse)
def home():
  return HTMLResponse(INDEX_HTML)

@app.post("/recommend")
async def recommend(file: UploadFile = File(...)):
    stage = "start"
    try:
        # 0) 이미지 로딩
        stage = "load_image"
        img = Image.open(io.BytesIO(await file.read())).convert("RGB")

        # 1) 텍스트 생성
        stage = "build_texts_for_image"
        cap_short, cap_combo, cap_long, emo = build_texts_for_image(img)

        # 2) Stage-1 점수 (BLIP2+CLIP cap_combo)
        stage = "stage1"
        s1 = score_all_songs_with_text(cap_combo)

        # 3) Stage-2 점수 (λ·s1 + (1-λ)·s2)
        stage = "stage2"
        lam = 0.0
        s2 = score_all_songs_with_text(cap_long)
        fused = lam * s1 + (1.0 - lam) * s2

        # 4) 상위 3개 뽑기 → JSON
        TOPK = 3
        top1 = topk_block(s1, k=TOPK)
        top2 = topk_block(fused, k=TOPK)

        payload = {
            "ok": True,
            "emotion": {"label": str(emo["label"]), "prob": float(_to_py(emo["prob"]))},
            "texts": {
                "blip2_caption": str(cap_short),
                "clip_plus_blip2": str(cap_combo),
                "instruct_expanded": str(cap_long),
            },
            "stage1": top1,
            "stage2": top2,
        }
        return JSONResponse(payload, media_type="application/json")
    except Exception as e:
        import traceback; traceback.print_exc()
        return JSONResponse({"ok": False, "stage": stage, "error": str(e)}, status_code=500)

print("✅ 4. FastAPI 웹 서버 및 Cloudflare Tunnel 실행 완료.")

In [ ]:
# --- 5. 서버 및 터널 실행 ---
PORT = 8001

# (1) cloudflared 설치
if not os.path.exists('/usr/local/bin/cloudflared'):
    print("Installing cloudflared...")
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb -O cloudflared.deb
    !sudo dpkg -i cloudflared.deb
    print("cloudflared installed.")

# (2) uvicorn 서버를 백그라운드 스레드에서 실행
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=PORT, log_level="info")

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
print(f"FastAPI server started in background on port {PORT}.")
time.sleep(4) # 서버가 시작될 때까지 잠시 대기

# (3) Cloudflare Tunnel을 백그라운드로 실행하고 로그 파일에 저장
# nohup: 터미널 연결이 끊겨도 프로세스를 계속 실행
# > cloudflared.log: 모든 출력을 cloudflared.log 파일에 저장
# 2>&1: 표준 오류도 표준 출력으로 리디렉션
# &: 백그라운드에서 실행
print("\nStarting Cloudflare Tunnel in the background...")
!nohup cloudflared tunnel --url http://localhost:{PORT} --no-autoupdate > cloudflared.log 2>&1 &
time.sleep(3) # 터널이 시작되고 로그가 기록될 때까지 잠시 대기
print("Cloudflare Tunnel is running.")

# (4) 로그 파일에서 퍼블릭 URL 추출
print("\nSearching for public URL in logs...")
public_url = None
for i in range(10): # 최대 10초간 URL을 찾기 위해 시도
    with open('cloudflared.log', 'r') as f:
        for line in f:
            match = re.search(r"https://[-a-zA-Z0-9]+\.trycloudflare\.com", line)
            if match:
                public_url = match.group(0)
                break
    if public_url:
        break
    time.sleep(1)

if public_url:
    print("\n" + "="*50)
    print(" ✅ Web Demo is Live!")
    print(f"👇👇👇 Click this public URL to access your demo 👇👇👇")
    print(f"      {public_url}")
    print("="*50)
else:
    print("\n❌ Could not find the public URL.")
    print("   Please check the 'cloudflared.log' file for errors:")
    !cat cloudflared.log

In [ ]:
# --- requirements.txt 생성 ---
print("Generating requirements.txt based on the current environment...")

!pip freeze | grep -E 'transformers|torch|pandas|numpy|Pillow|matplotlib|seaborn|scikit-learn|accelerate|bitsandbytes|ftfy|regex|librosa|huggingface-hub|fastapi|uvicorn|python-multipart|nest-asyncio|starlette|cloudflared|clip' > requirements.txt

print("\n--- Content of requirements.txt ---")
!cat requirements.txt

# 생성된 파일을 Google Drive에 저장
!cp requirements.txt "/content/drive/MyDrive/image-music-recommendation-system_final/"